<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Alpha_1%2B1_orderbook_Indicators_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model includes technical indicators RSI ATR OBV and LSTM model 1.0 to predict price in 15min 4H timeframes. Then it also has orderbook flow, liquidity zones, passive and active liquidity where the liquidity lies so you have information to make a decision for the trade.
it also analyze orderbook within 1 standard deviation to check limit orders volume in queue as it can influence the price movements in short terms.

in my opnion the only limitation currently, it has only data from kucoin. if we can fetch data from other exchanges, it will improve the efficiency.

In [ ]:
!pip install ccxt pandas numpy scikit-learn tensorflow pandas-ta matplotlib plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 4.9 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=80fc38b5612696cd1effec96904bb3c79cabaedd637bd615c83c9796eef92118
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
  Attempting uninstall: aiohttp


In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import ccxt
import logging
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize KuCoin Exchange
exchange = ccxt.kucoin()

# Fetch Historical Data (OHLCV)
def fetch_historical_data(symbol, timeframe, limit=500):
    """Fetch OHLCV data from KuCoin."""
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        logging.error(f"Error fetching historical data for {symbol}: {e}")
        return pd.DataFrame()

# Add Technical Indicators
def add_technical_indicators(df):
    """Add RSI, ATR, OBV, SMA, and EMA to the DataFrame."""
    df['RSI'] = df.ta.rsi(close='close', length=14)
    df['ATR'] = df.ta.atr(high='high', low='low', close='close', length=14)
    df['OBV'] = df.ta.obv(close='close', volume='volume')
    df['SMA_50'] = df.ta.sma(close='close', length=50)
    df['EMA_50'] = df.ta.ema(close='close', length=50)
    return df

# Fetch Level 2 Order Book
def fetch_order_book(symbol, depth=100):
    """Fetch bid-ask Level 2 order book data from KuCoin."""
    try:
        order_book = exchange.fetch_order_book(symbol, limit=depth)
        bids = order_book['bids'][:depth]  # Top bids
        asks = order_book['asks'][:depth]  # Top asks
        return bids, asks
    except Exception as e:
        logging.error(f"Error fetching order book for {symbol}: {e}")
        return [], []

# Identify High-Quality Liquidity Zones
def identify_liquidity_zones(bids, asks, volume_threshold=100, price_tolerance=0.001):
    """Identify high-quality liquidity zones (areas with significantly higher liquidity)."""
    try:
        # Extract prices and volumes
        bid_prices = [bid[0] for bid in bids]
        bid_volumes = [bid[1] for bid in bids]
        ask_prices = [ask[0] for ask in asks]
        ask_volumes = [ask[1] for ask in asks]

        # Function to group prices into zones
        def group_prices(prices, volumes, tolerance):
            zones = []
            current_zone = []
            current_price = None

            for price, volume in zip(prices, volumes):
                if current_price is None or abs(price - current_price) <= tolerance:
                    current_zone.append((price, volume))
                    current_price = price
                else:
                    zones.append(current_zone)
                    current_zone = [(price, volume)]
                    current_price = price

            if current_zone:
                zones.append(current_zone)
            return zones

        # Group bid and ask prices into zones
        bid_zones = group_prices(bid_prices, bid_volumes, price_tolerance)
        ask_zones = group_prices(ask_prices, ask_volumes, price_tolerance)

        # Filter zones based on volume threshold
        def filter_zones(zones, threshold):
            filtered = []
            for zone in zones:
                total_volume = sum(vol for _, vol in zone)
                if total_volume >= threshold:
                    avg_price = sum(price * vol for price, vol in zone) / total_volume
                    filtered.append((avg_price, total_volume))
            return filtered

        bid_liquidity_zones = filter_zones(bid_zones, volume_threshold)
        ask_liquidity_zones = filter_zones(ask_zones, volume_threshold)

        return bid_liquidity_zones, ask_liquidity_zones
    except Exception as e:
        logging.error(f"Error identifying liquidity zones: {e}")
        return [], []



# Fetch Executed Trade Data
def fetch_trade_data(symbol, since=None, limit=1000):
    """Fetch executed market orders from KuCoin."""
    try:
        trades = exchange.fetch_trades(symbol, since=since, limit=limit)
        df = pd.DataFrame(trades, columns=['timestamp', 'price', 'amount', 'side'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        logging.error(f"Error fetching trade data for {symbol}: {e}")
        return pd.DataFrame()

# Classify Active and Passive Liquidity
def classify_liquidity(trade_df, bids, asks, tolerance=0.001):
    """Classify trades as active liquidity (market orders) and passive liquidity (limit orders)."""
    if trade_df.empty or not bids or not asks:
        return trade_df

    best_bid = bids[0][0] if bids else None
    best_ask = asks[0][0] if asks else None

    if not best_bid or not best_ask:
        return trade_df

    # Classify active liquidity (market orders)
    trade_df['active_side'] = trade_df.apply(
        lambda row: 'buy' if abs(row['price'] - best_ask) <= best_ask * tolerance else (
            'sell' if abs(row['price'] - best_bid) <= best_bid * tolerance else None
        ),
        axis=1
    )

    # Weight active liquidity by trade size
    trade_df['weighted_amount'] = trade_df['amount'] * trade_df['price']
    return trade_df

# Analyze Active Liquidity
def analyze_active_liquidity(trade_df, timeframe):
    """Analyze active liquidity (aggressive buy/sell) over a given timeframe."""
    try:
        if trade_df.empty:
            raise ValueError("Trade data is empty.")

        trade_df = trade_df.set_index('timestamp')
        resampled = trade_df.resample(timeframe).agg({
            'weighted_amount': 'sum',  # Weighted by trade size
            'active_side': lambda x: (x == 'buy').sum()
        })

        resampled.rename(columns={'active_side': 'active_buy_count'}, inplace=True)
        resampled['active_sell_count'] = len(trade_df) - resampled['active_buy_count']

        resampled['active_buy_volume'] = trade_df[trade_df['active_side'] == 'buy'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()
        resampled['active_sell_volume'] = trade_df[trade_df['active_side'] == 'sell'].groupby(pd.Grouper(freq=timeframe))['weighted_amount'].sum()

        resampled['active_liquidity_ratio'] = (resampled['active_buy_volume'] - resampled['active_sell_volume']) / (
            resampled['active_buy_volume'] + resampled['active_sell_volume'] + 1e-9
        )  # Add small epsilon to avoid division by zero

        return resampled
    except Exception as e:
        logging.error(f"Error analyzing active liquidity for {timeframe}: {e}")
        return pd.DataFrame()

# Calculate Passive Liquidity Ratio
def calculate_passive_liquidity_ratio(bids, asks, current_price, std_dev):
    """Calculate passive liquidity ratio (bid/ask) within ±1 standard deviation."""
    price_range_low = current_price - std_dev
    price_range_high = current_price + std_dev

    # Filter bids and asks within the price range
    bid_volumes = [bid[1] for bid in bids if price_range_low <= bid[0] <= price_range_high]
    ask_volumes = [ask[1] for ask in asks if price_range_low <= ask[0] <= price_range_high]

    total_bid_volume = sum(bid_volumes)
    total_ask_volume = sum(ask_volumes)

    if total_bid_volume + total_ask_volume == 0:
        return 0  # Avoid division by zero

    passive_liquidity_ratio = (total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)
    return passive_liquidity_ratio

# Calculate Order Book Imbalance
def calculate_order_book_imbalance(bids, asks, current_price, std_dev):
    """Calculate order book imbalance within ±1 standard deviation."""
    price_range_low = current_price - std_dev
    price_range_high = current_price + std_dev

    # Filter bids and asks within the price range
    bid_volumes = [bid[1] for bid in bids if price_range_low <= bid[0] <= price_range_high]
    ask_volumes = [ask[1] for ask in asks if price_range_low <= ask[0] <= price_range_high]

    total_bid_volume = sum(bid_volumes)
    total_ask_volume = sum(ask_volumes)

    if total_bid_volume + total_ask_volume == 0:
        return 0  # Avoid division by zero

    order_book_imbalance = (total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)
    return order_book_imbalance

# Preprocess Data for LSTM
def preprocess_data(df, sequence_length=50):
    """Preprocess data for LSTM model."""
    scaler = MinMaxScaler()
    features = df[['close', 'RSI', 'ATR', 'OBV', 'SMA_50', 'EMA_50']].dropna()
    scaled_features = scaler.fit_transform(features)
    X, y = [], []
    for i in range(len(scaled_features) - sequence_length):
        X.append(scaled_features[i:i + sequence_length])
        y.append(scaled_features[i + sequence_length, 0])  # Predicting the 'close' price
    return np.array(X), np.array(y), scaler

# Build LSTM Model
def build_model(input_shape):
    """Build and compile the LSTM model."""
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Predict Price
def predict_price(symbol, sequence_length=50):
    """Predict price using LSTM model."""
    df_15m = fetch_historical_data(symbol, '15m', limit=500)
    df_4h = fetch_historical_data(symbol, '4h', limit=500)

    df_15m = add_technical_indicators(df_15m)
    df_4h = add_technical_indicators(df_4h)

    X_15m, _, scaler_15m = preprocess_data(df_15m, sequence_length)
    X_4h, _, scaler_4h = preprocess_data(df_4h, sequence_length)

    model_15m = build_model((X_15m.shape[1], X_15m.shape[2]))
    model_4h = build_model((X_4h.shape[1], X_4h.shape[2]))

    model_15m.fit(X_15m, X_15m[:, -1, 0], epochs=5, batch_size=32, verbose=1)
    model_4h.fit(X_4h, X_4h[:, -1, 0], epochs=5, batch_size=32, verbose=1)

    latest_sequence_15m = X_15m[-1].reshape(1, X_15m.shape[1], X_15m.shape[2])
    latest_sequence_4h = X_4h[-1].reshape(1, X_4h.shape[1], X_4h.shape[2])

    predicted_price_15m = scaler_15m.inverse_transform([[model_15m.predict(latest_sequence_15m)[0][0], 0, 0, 0, 0, 0]])[0][0]
    predicted_price_4h = scaler_4h.inverse_transform([[model_4h.predict(latest_sequence_4h)[0][0], 0, 0, 0, 0, 0]])[0][0]

    return predicted_price_15m, predicted_price_4h

# Main Function
def analyze_asset(symbol):
    """Analyze an asset using order flow, technical indicators, and LSTM predictions."""
    try:
        # Fetch data
        bids, asks = fetch_order_book(symbol, depth=100)
        trade_data = fetch_trade_data(symbol)
        df_15m = fetch_historical_data(symbol, '15m', limit=500)
        df_4h = fetch_historical_data(symbol, '4h', limit=500)

        if trade_data.empty or df_15m.empty or df_4h.empty:
            logging.warning(f"No data found for {symbol}. Skipping...")
            return

        # Add technical indicators
        df_15m = add_technical_indicators(df_15m)
        df_4h = add_technical_indicators(df_4h)

        # Classify active and passive liquidity
        trade_data = classify_liquidity(trade_data, bids, asks)

        # Analyze active liquidity
        active_liquidity_4h = analyze_active_liquidity(trade_data, '4h')
        active_liquidity_15m = analyze_active_liquidity(trade_data, '15min')

        # Calculate dynamic standard deviation
        std_dev = df_15m['close'].std()  # Standard deviation of 1-hour prices
        current_price = df_15m['close'].iloc[-1]

        # Calculate passive liquidity ratio
        passive_liquidity_ratio = calculate_passive_liquidity_ratio(bids, asks, current_price, std_dev)

        # Identify liquidity zones
        bid_liquidity_zones, ask_liquidity_zones = identify_liquidity_zones(bids, asks, volume_threshold=100, price_tolerance=0.001)

        # Calculate order book imbalance
        order_book_imbalance = calculate_order_book_imbalance(bids, asks, current_price, std_dev)


        # Predict prices
        predicted_price_15m, predicted_price_4h = predict_price(symbol)

        # Generate trading signal
        if predicted_price_15m > current_price * 1.002:  # 0.2% threshold
            signal = "🚀 BUY Signal"
        elif predicted_price_15m < current_price * 0.998:  # 0.2% threshold
            signal = "🔻 SELL Signal"
        else:
            signal = "❌ NO TRADE"

        # Output results
        print(f"\n🔹 {symbol}")
        print(f"📈 Current Price: {current_price:.2f}")
        print(f"📊 Trading Signal: {signal}")
        print(f"📊 15M Prediction: {predicted_price_15m:.2f}")
        print(f"📊 4H Prediction: {predicted_price_4h:.2f}")
        print(f"📊 4H Active Liquidity Ratio: {active_liquidity_4h['active_liquidity_ratio'].iloc[-1]:.2f}")
        print(f"📊 15M Active Liquidity Ratio: {active_liquidity_15m['active_liquidity_ratio'].iloc[-1]:.2f}")
        print(f"📊 Passive Liquidity Ratio: {passive_liquidity_ratio:.2f}")
        print(f"📊 Order Book Imbalance: {order_book_imbalance:.2f}")
        print(f"📊 Bid Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in bid_liquidity_zones]}")
        print(f"📊 Ask Liquidity Zones: {[f'{price:.4f} ({vol:.2f})' for price, vol in ask_liquidity_zones]}")
        print(f"📊 RSI: {df_15m['RSI'].iloc[-1]:.2f}")
        print(f"📊 ATR: {df_15m['ATR'].iloc[-1]:.2f}")
        print(f"📊 OBV: {df_15m['OBV'].iloc[-1]:.2f}")
    except Exception as e:
        logging.error(f"\n🔹 {symbol}")
        logging.error(f"⚠️ Error: {e}")

# List of Assets to Analyze
assets = ['BTC/USDT', 'ETH/USDT', 'ADA/USDT', 'FET/USDT', 'NEAR/USDT', 'AVAX/USDT', 'DOGE/USDT',
          'XRP/USDT', 'LINK/USDT', 'INJ/USDT', 'HNT/USDT', 'CRV/USDT', 'SOL/USDT', 'LTC/USDT',
          'MNT/USDT', 'AAVE/USDT', 'JUP/USDT']

# Analyze each asset
for asset in assets:
    analyze_asset(asset)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.1882
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0113
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0060
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0037
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0028
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.1059
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0158
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0086
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0069
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0045
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step

🔹 BTC/USDT
📈 Current Price: 85800.70
📊 Trading Signal: 🔻 SELL Signal
📊 15M Prediction: 85088.02
📊 4H Prediction: 89231.00
📊 4H Active Liquidity Ratio: nan
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: -0.06
📊 Order Book Imbalance: -0.06
📊 Bid Liquidity Zones: []
📊 A

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.2079
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0095
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0054
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0035
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0028
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.0866
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0108
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0057
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0034
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0028
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step

🔹 ETH/USDT
📈 Current Price: 2337.08
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 2336.25
📊 4H Prediction: 2450.68
📊 4H Active Liquidity Ratio: nan
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: -0.02
📊 Order Book Imbalance: -0.02
📊 Bid Liquidity Zones: []
📊 Ask Liq

...

13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.0896
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0132
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0046
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0034
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0026
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.2965
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0172
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0113
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0094
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0079
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step

🔹 MNT/USDT
📈 Current Price: 0.76
📊 Trading Signal: 🚀 BUY Signal
📊 15M Prediction: 0.77
📊 4H Prediction: 0.85
📊 4H Active Liquidity Ratio: 0.19
📊 15M Active Liquidity Ratio: -0.12
📊 Passive Liquidity Ratio: 0.01
📊 Order Book Imbalance: 0.01
📊 Bid Liquidity Zones: ['0.7603 (617.70)'

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.1464
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0146
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0050
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0030
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0021
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0766
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0180
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0113
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0083
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0059
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step

🔹 AAVE/USDT
📈 Current Price: 205.69
📊 Trading Signal: ❌ NO TRADE
📊 15M Prediction: 205.74
📊 4H Prediction: 207.68
📊 4H Active Liquidity Ratio: nan
📊 15M Active Liquidity Ratio: nan
📊 Passive Liquidity Ratio: -0.11
📊 Order Book Imbalance: -0.11
📊 Bid Liquidity Zones: []
📊 Ask Liquidit

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.1669
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0111
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0069
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0049
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0038
Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0871
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0204
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0129
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0100
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0070
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step

🔹 JUP/USDT
📈 Current Price: 0.75
📊 Trading Signal: 🔻 SELL Signal
📊 15M Prediction: 0.73
📊 4H Prediction: 0.75
📊 4H Active Liquidity Ratio: 0.80
📊 15M Active Liquidity Ratio: 0.90
📊 Passive Liquidity Ratio: 0.10
📊 Order Book Imbalance: 0.10
📊 Bid Liquidity Zones: ['0.7439 (260741.9